In [1]:
import csv 
import pandas as pd
import dask.dataframe as dd

filename = '/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/cdr/cdr_regiao_imediata_jf_all.txt'

cdr = dd.read_csv(filename, delimiter=';')

In [2]:
cdr_new = cdr[cdr['duration'].between(0.07,120)]

In [3]:
def filters_of_call(cdr):
    cdr_filter_by_quantity_of_call = cdr.groupby('user_from').size().reset_index().rename(columns={0:'QUANTITY_OF_CALLS'})

    cdr_filter_by_quantity_of_call = cdr_filter_by_quantity_of_call.loc[(
                                    (cdr_filter_by_quantity_of_call['QUANTITY_OF_CALLS'] >= 3) 
                                    & (cdr_filter_by_quantity_of_call['QUANTITY_OF_CALLS'] <= 500))]
    return cdr

def cdr_filter_by_quantity_of_days(cdr_new):
    cdr_filter_by_quantity_of_days = cdr_new.groupby('user_from').date.nunique().to_frame('QUANTITY_OF_DAYS').reset_index()
    cdr_filter_by_quantity_of_days = cdr_filter_by_quantity_of_days.loc[(cdr_filter_by_quantity_of_days['QUANTITY_OF_DAYS'] >= 3)]
    
#     df_filter_over_call = dd.merge(cdr_filter_by_quantity_of_call, cdr_filter_by_quantity_of_days,
#                                    how='inner',on=['user_from'])
#     cdr_after_filters_of_calls = cdr_new[(cdr_new['user_from'].isin(df_filter_over_call['user_from']))]
#     cdr_after_filters_of_calls.to_csv('file.csv')
    return cdr_filter_by_quantity_of_days

In [4]:
cdr_call = filters_of_call(cdr_new)
cdr_days = cdr_filter_by_quantity_of_days(cdr_new)

In [5]:
df_filter_over_call = dd.merge(cdr_call, cdr_days, how='inner',on=['user_from'])

In [ ]:
cdr_after_filters_of_calls = cdr_new[(cdr_new['user_from'].isin(df_filter_over_call['user_from']))]